# Axioms

In [2]:
from pref_voting.profiles_with_ties import *
from pref_voting.profiles import *
from pref_voting.voting_methods import *
from functools import partial
from multiprocess import Pool, cpu_count
from tqdm.notebook import tqdm

import pandas as pd

from pref_voting.generate_profiles import generate_profile, generate_truncated_profile

import pref_voting.profiles

In [3]:
class Axiom(object): 
    def __init__(self, name, has_violation, find_all_violations):
        self.name = name
        self.has_violation = has_violation
        self.find_all_violations = find_all_violations
        self.satisfying_vms = list()
        self.violating_vms = list()
        
    def satisfies(vm): 
        return vm.name in self.satisfying_vms
        
    def violates(vm): 
        return vm.name in self.violating_vms
        
    def add_satisfying_vms(vms): 
        self.satisfying_vms += v

    def add_violating_vms(vms): 
        self.violating_vms += v


In [4]:
# Dominance notions

def display_mg(edata): 
    if type(edata) == Profile or type(edata) == ProfileWithTies: 
        edata.display_margin_graph()
    else: 
        edata.display()


def list_to_string(cands, cmap): 
    return "{" + ', '.join([cmap[c] for c in cands]) + "}"

def has_pareto_violation(edata, vm, verbose=False):
    """
    Returns True if there is some winner according to vm is Pareto dominated (there is a candidate that is unanimously preferred to the winner).
    
    Args:
        edata (Profile, ProfileWithTies): the election data.
        vm (VotingMethod): A voting method to test.
        verbose (bool, default=False): If a violation is found, display the violation. 

    Returns: 
        Result of the test (bool): Returns True if there is a violation and False otherwise. 

    """

    ws = vm(edata)
    for w in ws: 
        for c in edata.candidates: 
            if edata.margin(c,w)==edata.num_voters:
                if verbose:  
                    edata.display()
                    print(edata.description())
                    vm.display(edata)
                    print(f"The winner {w} is Pareto dominated by {c}. ")
                    print()
                return True
    return False

def find_all_pareto_violations(edata, vm, verbose=False):
    """
    Return all Pareto dominated candidates.
    
    Args:
        edata (Profile, ProfileWithTies): the election data.
        vm (VotingMethod): A voting method to test.
        verbose (bool, default=False): If a violation is found, display the violation. 

    Returns: 
        pareto_dominated_winners: A list of tuples of candidates (w, c) where w is a winner according to vm and Pareto dominated by c (and the empty list if there are none). 

    """

    ws = vm(edata)
    pareto_dominated_winners = list()
    for w in ws: 
        for c in edata.candidates: 
            if edata.margin(c,w)==edata.num_voters:
                pareto_dominated_winners.append((w, c))
    
    if len(pareto_dominated_winners) > 0 and verbose: 
        edata.display()
        print(edata.description())
        vm.display(edata)
        for w,c in pareto_dominated_winners:
            print(f"The winner {w} is Pareto dominated by {c}. ")

    return pareto_dominated_winners

pareto = Axiom(
    "Pareto",
    has_violation = has_pareto_violation,
    find_all_violations = find_all_pareto_violations, 
)

def has_condorcet_winner_violation(edata, vm, verbose=False):
    """
    Returns True if there is a Condorcet winner in edata (a candidate that is majority preferred to every other candidate) that is not the unique winner according to vm.
    
    Args:
        edata (Profile, ProfileWithTies, MajorityGraph, or MarginGraph): the election data.
        vm (VotingMethod): A voting method to test.
        verbose (bool, default=False): If a violation is found, display the violation. 

    Returns: 
        Result of the test (bool): Returns True if there is a violation and False otherwise. 

    """

    cw = edata.condorcet_winner()

    ws = vm(edata)

    if cw is not None and ws != [cw]:
        if verbose: 
            if type(edata) == Profile or type(edata) == ProfileWithTies: 
                edata.display_margin_graph()
            else: 
                edata.display()
            print(edata.description())
            print(f"The Condorcet winner {cw} is not the unique winner: ")
            vm.display(edata)
        return True 
    return False

def find_condorcet_winner_violation(edata, vm, verbose=False):
    """
    Returns the Condorcet winner that is not the unique winner according to vm.
    
    Args:
        edata (Profile, ProfileWithTies, MajorityGraph, or MarginGraph): the election data.
        vm (VotingMethod): A voting method to test.
        verbose (bool, default=False): If a violation is found, display the violation. 

    Returns: 
        Result of the test (bool): Returns True if there is a violation and False otherwise. 

    """

    cw = edata.condorcet_winner()

    ws = vm(edata)

    if cw is not None and ws != [cw]:
        if verbose: 
            if type(edata) == Profile or type(edata) == ProfileWithTies: 
                edata.display_margin_graph()
            else: 
                edata.display()
            print(edata.description())
            print(f"The Condorcet winner {cw} is not the unique winner: ")
            vm.display(edata)
        return [cw] 
    return list()

condorcet_winner = Axiom(
    "Condorcet Winner",
    has_violation = has_condorcet_winner_violation,
    find_all_violations = find_condorcet_winner_violation, 
)


def has_condorcet_loser_violation(edata, vm, verbose=False):
    """
    Returns True if there is a winner according to vm that is a Condorcet loser (a candidate that loses head-to-head to every other candidate).  
    
    Args:
        edata (Profile, ProfileWithTies, MajorityGraph, or MarginGraph): the election data.
        vm (VotingMethod): A voting method to test.
        verbose (bool, default=False): If a violation is found, display the violation. 

    Returns: 
        Result of the test (bool): Returns True if there is a violation and False otherwise. 

    """

    cl = edata.condorcet_loser()

    ws = vm(edata)

    if cl is not None and cl in ws:
        if verbose: 
            display_mg(edata.display)
            print(edata.description())
            print(f"The Condorcet loser {cl} is an element of the winning set: ")
            vm.display(edata)
        return True 
    return False

def find_condorcet_loser_violation(edata, vm, verbose=False):
    """
    Returns the Condorcet loser (a candidate that loses head-to-head to every other candidate) who is a winner according to vm.  
    
    Args:
        edata (Profile, ProfileWithTies, MajorityGraph, or MarginGraph): the election data.
        vm (VotingMethod): A voting method to test.
        verbose (bool, default=False): If a violation is found, display the violation. 

    Returns: 
        Result of the test (bool): Returns True if there is a violation and False otherwise. 

    """

    cl = edata.condorcet_loser()

    ws = vm(edata)

    if cl is not None and cl in ws:
        if verbose: 
            display_mg(edata.display)
            print(edata.description())
            print(f"The Condorcet loser {cl} is an element of the winning set: ")
            vm.display(edata)
        return [cl] 
    return list()

condorcet_loser = Axiom(
    "Condorcet Loser",
    has_violation = has_condorcet_loser_violation,
    find_all_violations = find_condorcet_winner_violation, 
)


def has_smith_violation(edata, vm, verbose=False):
    """
    Returns True if there is a winner according to vm that is not in the Smith set (the smallest set of candidates such that every candidate in the set is majority preferred to every candidate outside the set).  
    
    Args:
        edata (Profile, ProfileWithTies, MajorityGraph, or MarginGraph): the election data.
        vm (VotingMethod): A voting method to test.
        verbose (bool, default=False): If a violation is found, display the violation. 

    Returns: 
        Result of the test (bool): Returns True if there is a violation and False otherwise. 

    """

    s_set = smith_set(edata)
    ws = vm(edata)

    winners_not_in_smith = [w not in s_set for w in ws]
    if len(winners_not_in_smith) > 0: 
        if verbose:
            display_mg(edata.display)
            print(f"The winners that are not in the Smith set: {list_to_string(winners_not_in_smith, edata.cmap)}.")
            vm.display(edata)
        return True 
    return False

def find_all_smith_violations(edata, vm, verbose=False):
    """
    Returns the winners according to vm that are not in the Smith set (the smallest set of candidates such that every candidate in the set is majority preferred to every candidate outside the set).  
    
    Args:
        edata (Profile, ProfileWithTies, MajorityGraph, or MarginGraph): the election data.
        vm (VotingMethod): A voting method to test.
        verbose (bool, default=False): If a violation is found, display the violation. 

    Returns: 
        Result of the test (bool): Returns True if there is a violation and False otherwise. 

    """

    s_set = smith_set(edata)
    ws = vm(edata)

    winners_not_in_smith = [w not in s_set for w in ws]
    if len(winners_not_in_smith) > 0: 
        if verbose:
            display_mg(edata.display)
            print(f"The winners that are not in the Smith set: {list_to_string(winners_not_in_smith, edata.cmap)}.")
            vm.display(edata)
        return winners_not_in_smith 
    return list()

smith = Axiom(
    "Smith",
    has_violation = has_smith_violation,
    find_all_violations = find_all_smith_violations, 
)

def has_schwartz_violation(edata, vm, verbose=False):
    """
    Returns True if there is a winner according to vm that is not in the Schwartz set (the set of all candidates x such that if y can reach x in the transitive closer of the majority relation, then x can reach y in the transitive closer of the majority relation.).  
    
    Args:
        edata (Profile, ProfileWithTies, MajorityGraph, or MarginGraph): the election data.
        vm (VotingMethod): A voting method to test.
        verbose (bool, default=False): If a violation is found, display the violation. 

    Returns: 
        Result of the test (bool): Returns True if there is a violation and False otherwise. 

    """

    s_set = schwartz_set(edata)
    ws = vm(edata)

    winners_not_in_schwartz = [w not in s_set for w in ws]
    if len(winners_not_in_schwartz) > 0: 
        if verbose:
            display_mg(edata.display)
            print(f"The winners that are not in the Schwartz set: {list_to_string(winners_not_in_schwartz, edata.cmap)}.")
            vm.display(edata)
        return True 
    return False

def find_all_schwartz_violations(edata, vm, verbose=False):
    """
    Returns the winners according to vm that are not in the Schwartz set (the set of all candidates x such that if y can reach x in the transitive closer of the majority relation, then x can reach y in the transitive closer of the majority relation).  
    
    Args:
        edata (Profile, ProfileWithTies, MajorityGraph, or MarginGraph): the election data.
        vm (VotingMethod): A voting method to test.
        verbose (bool, default=False): If a violation is found, display the violation. 

    Returns: 
        Result of the test (bool): Returns True if there is a violation and False otherwise. 

    """

    s_set = schwartz_set(edata)
    ws = vm(edata)

    winners_not_in_schwartz = [w not in s_set for w in ws]
    if len(winners_not_in_schwartz) > 0: 
        if verbose:
            display_mg(edata.display)
            print(f"The winners that are not in the Schwartz set: {list_to_string(winners_not_in_schwartz, edata.cmap)}.")
            vm.display(edata)
        return winners_not_in_schwartz 
    return list()

schwartz = Axiom(
    "Schwartz",
    has_violation = has_schwartz_violation,
    find_all_violations = find_all_schwartz_violations, 
)





In [5]:
print(has_smith_violation)

<function has_smith_violation at 0x2a06ad990>


In [6]:
prof = generate_profile(4, 6)
prof.display()
print(prof)
print(prof.description())

+---+---+---+---+---+---+
| 1 | 1 | 1 | 1 | 1 | 1 |
+---+---+---+---+---+---+
| 3 | 0 | 0 | 0 | 3 | 1 |
| 2 | 3 | 1 | 2 | 1 | 0 |
| 1 | 2 | 2 | 3 | 2 | 2 |
| 0 | 1 | 3 | 1 | 0 | 3 |
+---+---+---+---+---+---+
+---+---+---+---+---+---+
| 1 | 1 | 1 | 1 | 1 | 1 |
+---+---+---+---+---+---+
| 3 | 0 | 0 | 0 | 3 | 1 |
| 2 | 3 | 1 | 2 | 1 | 0 |
| 1 | 2 | 2 | 3 | 2 | 2 |
| 0 | 1 | 3 | 1 | 0 | 3 |
+---+---+---+---+---+---+
Profile([[3, 2, 1, 0], [0, 3, 2, 1], [0, 1, 2, 3], [0, 2, 3, 1], [3, 1, 2, 0], [1, 0, 2, 3]], rcounts=[1, 1, 1, 1, 1, 1], cmap={0: '0', 1: '1', 2: '2', 3: '3'})


In [7]:
prof = generate_truncated_profile(4, 6)
prof.display()
print(prof)
print(prof.description())

+---+---+---+---+---+---+
| 1 | 1 | 1 | 1 | 1 | 1 |
+---+---+---+---+---+---+
| 3 | 3 | 2 | 1 | 1 | 3 |
|   | 2 | 1 | 3 | 2 | 2 |
|   | 0 |   | 0 | 3 | 1 |
|   |   |   |   |   |   |
+---+---+---+---+---+---+
ProfileWithTies([{3: 1}, {3: 1, 2: 2, 0: 3}, {2: 1, 1: 2}, {1: 1, 3: 2, 0: 3}, {1: 1, 2: 2, 3: 3}, {3: 1, 2: 2, 1: 3}], rcounts=[1, 1, 1, 1, 1, 1], cmap={0: '0', 1: '1', 2: '2', 3: '3'})


In [27]:

def record_axiom_violation_data(
    axioms, 
    vms, 
    num_cands, 
    num_voters, 
    probmod, 
    verbose, 
    t
    ):

    prof = generate_profile(num_cands, num_voters, probmod=probmod)
    
    return {ax.name: {vm.name: ax.has_violation(prof, vm, verbose=verbose) for vm in vms} for ax in axioms}

def axiom_violations_data(
    axioms,
    vms,
    numbers_of_candidates=[3, 4, 5],
    numbers_of_voters=[4, 5, 10, 11,  20, 21, 50, 51,  100, 101,  500, 501,  1000, 1001],
    probmods=["IC"],
    num_trials=10000,
    verbose=False,
    use_parallel=True,
    num_cpus=12,
):
    """
    Returns a Pandas DataFrame with the Condorcet efficiency of a list of voting methods.

    Args:
        vms (list(functions)): A list of voting methods,
        numbers_of_candidates (list(int), default = [3, 4, 5]): The numbers of candidates to check.
        numbers_of_voters (list(int), default = [5, 25, 50, 100]): The numbers of voters to check.
        probmod (str, default="IC"): The probability model to be passed to the ``generate_profile`` method
        num_trials (int, default=10000): The number of profiles to check for different winning sets.
        use_parallel (bool, default=True): If True, then use parallel processing.
        num_cpus (int, default=12): The number of (virtual) cpus to use if using parallel processing.

    """

    if use_parallel:
        pool = Pool(num_cpus)

    data_for_df = {
        "axiom": list(),
        "vm": list(),
        "num_cands": list(),
        "num_voters": list(),
        "probmod": list(),
        "num_trials": list(),
        "num_violations": list(),
    }
    for probmod in probmods:
        print(f"{probmod} probability model")
        for num_cands in tqdm(numbers_of_candidates, leave=False):
            for num_voters in tqdm(numbers_of_voters, leave=False):
                #print(f"{num_cands} candidates, {num_voters} voters")
                _verbose = verbose if not use_parallel else False
                get_data = partial(
                    record_axiom_violation_data,
                    axioms,
                    vms,
                    num_cands,
                    num_voters,
                    probmod,
                    _verbose
                )

                if use_parallel:
                    data = pool.map(get_data, range(num_trials))
                else:
                    data = list(map(get_data, range(num_trials)))

                for ax in axioms: 
                    for vm in vms: 
                        data_for_df["axiom"].append(ax.name)
                        data_for_df["vm"].append(vm.name)
                        data_for_df["num_cands"].append(num_cands)
                        data_for_df["num_voters"].append(num_voters)
                        data_for_df["probmod"].append(probmod)
                        data_for_df["num_trials"].append(num_trials)
                        data_for_df["num_violations"].append(sum([d[ax.name][vm.name] for d in data]))
    print("Done.")
    return pd.DataFrame(data_for_df)





In [28]:
df = axiom_violations_data([ pareto ], [plurality, getcha, split_cycle_faster], num_trials=100, use_parallel=False, verbose=True)
df

IC probability model


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/14 [00:00<?, ?it/s]

+---+---+
| 2 | 2 |
+---+---+
| 0 | 1 |
| 1 | 2 |
| 2 | 0 |
+---+---+
Profile([[0, 1, 2], [1, 2, 0]], rcounts=[2, 2], cmap={0: '0', 1: '1', 2: '2'})
GETCHA winners are {0, 1, 2}
The winner 2 is Pareto dominated by 1. 

+---+---+
| 2 | 2 |
+---+---+
| 2 | 0 |
| 1 | 2 |
| 0 | 1 |
+---+---+
Profile([[2, 1, 0], [0, 2, 1]], rcounts=[2, 2], cmap={0: '0', 1: '1', 2: '2'})
GETCHA winners are {0, 1, 2}
The winner 1 is Pareto dominated by 2. 



  0%|          | 0/14 [00:00<?, ?it/s]

+---+---+---+
| 2 | 1 | 1 |
+---+---+---+
| 1 | 3 | 3 |
| 0 | 1 | 0 |
| 2 | 2 | 1 |
| 3 | 0 | 2 |
+---+---+---+
Profile([[1, 0, 2, 3], [3, 1, 2, 0], [3, 0, 1, 2]], rcounts=[2, 1, 1], cmap={0: '0', 1: '1', 2: '2', 3: '3'})
GETCHA winners are {0, 1, 2, 3}
The winner 2 is Pareto dominated by 1. 

+---+---+---+---+
| 1 | 1 | 1 | 1 |
+---+---+---+---+
| 0 | 1 | 2 | 1 |
| 2 | 3 | 0 | 3 |
| 1 | 2 | 1 | 0 |
| 3 | 0 | 3 | 2 |
+---+---+---+---+
Profile([[0, 2, 1, 3], [1, 3, 2, 0], [2, 0, 1, 3], [1, 3, 0, 2]], rcounts=[1, 1, 1, 1], cmap={0: '0', 1: '1', 2: '2', 3: '3'})
GETCHA winners are {0, 1, 2, 3}
The winner 3 is Pareto dominated by 1. 

+---+---+---+
| 2 | 1 | 1 |
+---+---+---+
| 1 | 0 | 2 |
| 3 | 2 | 1 |
| 2 | 1 | 0 |
| 0 | 3 | 3 |
+---+---+---+
Profile([[1, 3, 2, 0], [0, 2, 1, 3], [2, 1, 0, 3]], rcounts=[2, 1, 1], cmap={0: '0', 1: '1', 2: '2', 3: '3'})
GETCHA winners are {0, 1, 2, 3}
The winner 3 is Pareto dominated by 1. 

+---+---+---+
| 2 | 1 | 1 |
+---+---+---+
| 3 | 1 | 2 |
| 0 | 2 | 

  0%|          | 0/14 [00:00<?, ?it/s]

+---+---+---+---+
| 1 | 1 | 1 | 1 |
+---+---+---+---+
| 0 | 3 | 1 | 4 |
| 4 | 1 | 3 | 3 |
| 3 | 2 | 0 | 1 |
| 1 | 0 | 2 | 2 |
| 2 | 4 | 4 | 0 |
+---+---+---+---+
Profile([[0, 4, 3, 1, 2], [3, 1, 2, 0, 4], [1, 3, 0, 2, 4], [4, 3, 1, 2, 0]], rcounts=[1, 1, 1, 1], cmap={0: '0', 1: '1', 2: '2', 3: '3', 4: '4'})
GETCHA winners are {0, 1, 2, 3, 4}
The winner 2 is Pareto dominated by 1. 

+---+---+---+---+
| 1 | 1 | 1 | 1 |
+---+---+---+---+
| 1 | 0 | 2 | 4 |
| 0 | 2 | 3 | 2 |
| 4 | 1 | 0 | 0 |
| 2 | 4 | 1 | 3 |
| 3 | 3 | 4 | 1 |
+---+---+---+---+
Profile([[1, 0, 4, 2, 3], [0, 2, 1, 4, 3], [2, 3, 0, 1, 4], [4, 2, 0, 3, 1]], rcounts=[1, 1, 1, 1], cmap={0: '0', 1: '1', 2: '2', 3: '3', 4: '4'})
GETCHA winners are {0, 1, 2, 3, 4}
The winner 3 is Pareto dominated by 2. 

+---+---+---+---+
| 1 | 1 | 1 | 1 |
+---+---+---+---+
| 2 | 4 | 0 | 3 |
| 3 | 0 | 3 | 1 |
| 0 | 3 | 2 | 4 |
| 1 | 2 | 1 | 2 |
| 4 | 1 | 4 | 0 |
+---+---+---+---+
Profile([[2, 3, 0, 1, 4], [4, 0, 3, 2, 1], [0, 3, 2, 1, 4], [3, 1, 4

,axiom,vm,num_cands,num_voters,probmod,num_trials,num_violations
0,Pareto Criterion,Plurality,3,4,IC,100,0
1,Pareto Criterion,GETCHA,3,4,IC,100,2
2,Pareto Criterion,Split Cycle,3,4,IC,100,0
3,Pareto Criterion,Plurality,3,5,IC,100,0
4,Pareto Criterion,GETCHA,3,5,IC,100,0
...,...,...,...,...,...,...,...
121,Pareto Criterion,GETCHA,5,1000,IC,100,0
122,Pareto Criterion,Split Cycle,5,1000,IC,100,0
123,Pareto Criterion,Plurality,5,1001,IC,100,0
124,Pareto Criterion,GETCHA,5,1001,IC,100,0


In [4]:
num_cands = 4
num_voters = 3
for t in range(1000): 
    prof = generate_profile(num_cands, num_voters)
    if has_smith_violation(prof, smith_set, verbose=True):
        prof.display()
        #prof.display_margin_graph()


The winner 0 is Pareto dominated by 3. 
Smith Set winners are {0, 1, 2, 3}
+---+---+---+
| 1 | 1 | 1 |
+---+---+---+
| 2 | 1 | 3 |
| 1 | 3 | 0 |
| 3 | 0 | 2 |
| 0 | 2 | 1 |
+---+---+---+
The winner 2 is Pareto dominated by 3. 
Smith Set winners are {0, 1, 2, 3}
+---+---+---+
| 1 | 1 | 1 |
+---+---+---+
| 0 | 3 | 1 |
| 1 | 2 | 3 |
| 3 | 0 | 2 |
| 2 | 1 | 0 |
+---+---+---+
The winner 1 is Pareto dominated by 0. 
Smith Set winners are {0, 1, 2, 3}
+---+---+---+
| 1 | 1 | 1 |
+---+---+---+
| 2 | 0 | 3 |
| 0 | 1 | 2 |
| 1 | 3 | 0 |
| 3 | 2 | 1 |
+---+---+---+
The winner 1 is Pareto dominated by 3. 
Smith Set winners are {0, 1, 2, 3}
+---+---+---+
| 1 | 1 | 1 |
+---+---+---+
| 0 | 3 | 2 |
| 3 | 1 | 0 |
| 1 | 2 | 3 |
| 2 | 0 | 1 |
+---+---+---+
The winner 1 is Pareto dominated by 2. 
Smith Set winners are {0, 1, 2, 3}
+---+---+---+
| 1 | 1 | 1 |
+---+---+---+
| 2 | 3 | 0 |
| 1 | 0 | 2 |
| 3 | 2 | 1 |
| 0 | 1 | 3 |
+---+---+---+
The winner 0 is Pareto dominated by 2. 
Smith Set winners are {0,